In [2]:
import pandas as pd
import os

# --- 1. ファイルパスの設定 ---
GROUND_TRUTH_CSV = '../data/ground_truth/annotation_target_list.csv'
LLM_PREDICTIONS_CSV = '../data/processed/prediction_llm.csv'

# --- 2. データの読み込みと結合 ---
try:
    df_gt = pd.read_csv(GROUND_TRUTH_CSV)
    df_llm = pd.read_csv(LLM_PREDICTIONS_CSV)

    # 評価対象のモデルのカラム名
    best_model_column = 'prediction_rule3_gemini-2_5-flash' # Few-shot版がF1最高だったのでこれを採用

    df_review = pd.merge(
        df_gt[['citing_paper_doi', 'citing_paper_title', 'is_data_used_gt']],
        df_llm[['citing_paper_doi', best_model_column]],
        on='citing_paper_doi',
        how='inner'
    )
    
    # -1やNaNを除外
    df_review.dropna(inplace=True)
    df_review = df_review[df_review[best_model_column] != -1]
    df_review[best_model_column] = df_review[best_model_column].astype(int)
    df_review['is_data_used_gt'] = df_review['is_data_used_gt'].astype(int)
    
    print("✅ 正解データとLLM予測結果の読み込み・結合が完了しました。")

except FileNotFoundError as e:
    print(f"❌ エラー: ファイルが見つかりません。パスを確認してください。 {e}")
    df_review = pd.DataFrame()

# --- 3. 判断が食い違った論文を抽出して表示 ---
if not df_review.empty:
    
    # 判断の不一致を検出 (正解ラベルと予測が異なるもの)
    disagreements = df_review[df_review['is_data_used_gt'] != df_review[best_model_column]].copy()

    # 人間の判断とLLMの判断を分かりやすくラベル付け
    disagreements['Human_Label'] = disagreements['is_data_used_gt'].map({1: 'Used', 0: 'Not Used'})
    disagreements['LLM_Prediction'] = disagreements[best_model_column].map({1: 'Used', 0: 'Not Used'})

    print("\n" + "="*70)
    print("【人間の判断とLLMの判断が食い違った「怪しい」論文リスト】")
    print("="*70)
    
    if not disagreements.empty:
        print(f"合計 {len(disagreements)} 件の食い違いが見つかりました。")
        # to_string()ですべての行を省略せずに表示
        print(disagreements[['citing_paper_doi', 'citing_paper_title', 'Human_Label', 'LLM_Prediction']].to_string())
    else:
        print("✅ 人間の判断とLLMの判断はすべて一致していました。")
        
else:
    print("評価対象のデータがありませんでした。")

✅ 正解データとLLM予測結果の読み込み・結合が完了しました。

【人間の判断とLLMの判断が食い違った「怪しい」論文リスト】
合計 35 件の食い違いが見つかりました。
                         citing_paper_doi                                                                                                                                                                                        citing_paper_title Human_Label LLM_Prediction
1          10.1016/j.jaridenv.2024.105282                                                                                                     Potential effects of climate change on cacti distribution and conservation in North American drylands        Used       Not Used
2          10.1016/j.revpalbo.2023.104989                                                                                                                                     Approaches to pollen taxonomic harmonisation in Quaternary palynology    Not Used           Used
9         10.1016/j.resconrec.2023.107296                                                            

In [4]:
disagreements

,citing_paper_eid_gt,citing_paper_doi,citing_paper_title_gt,cited_data_paper_title_gt,is_data_used_gt,citing_paper_eid,citing_paper_title,cited_data_paper_title,prediction_rule3_abstract,prediction_rule3_fulltext,prediction_rule3_fulltext_few_shot,prediction_rule3_gemini-2_5-flash,prediction_rule3_gemini-2_5-flash_zeroshot,citing_paper_eid_text,citing_paper_title_text,cited_data_paper_title_text,abstract,full_text
1,2-s2.0-85210281314,10.1016/j.jaridenv.2024.105282,Potential effects of climate change on cacti d...,"The World Checklist of Vascular Plants, a cont...",1,2-s2.0-85210281314,Potential effects of climate change on cacti d...,"The World Checklist of Vascular Plants, a cont...",0,1,0,0,1,2-s2.0-85210281314,Potential effects of climate change on cacti d...,"The World Checklist of Vascular Plants, a cont...",Climate change is expected to adversely impact...,Drylands biodiversity is essential to the well...
2,2-s2.0-85171680307,10.1016/j.revpalbo.2023.104989,Approaches to pollen taxonomic harmonisation i...,European pollen-based REVEALS land-cover recon...,0,2-s2.0-85171680307,Approaches to pollen taxonomic harmonisation i...,European pollen-based REVEALS land-cover recon...,0,0,0,1,0,2-s2.0-85171680307,Approaches to pollen taxonomic harmonisation i...,European pollen-based REVEALS land-cover recon...,Pollen taxonomic harmonisation involves the st...,Quaternary pollen analysis demands careful and...
9,2-s2.0-85176563605,10.1016/j.resconrec.2023.107296,Government resource allocation practices towar...,Global Carbon Budget 2022,1,2-s2.0-85176563605,Government resource allocation practices towar...,Global Carbon Budget 2022,0,0,0,0,1,2-s2.0-85176563605,Government resource allocation practices towar...,Global Carbon Budget 2022,Government resource allocation practices for a...,The Chinese central government uses a top-down...
10,2-s2.0-105002637578,10.1016/j.solener.2025.113509,Predictive modeling and optimization of CIGS t...,Modeling and performance analysis dataset of a...,1,2-s2.0-105002637578,Predictive modeling and optimization of CIGS t...,Modeling and performance analysis dataset of a...,0,0,0,0,0,2-s2.0-105002637578,Predictive modeling and optimization of CIGS t...,Modeling and performance analysis dataset of a...,This study employs Machine Learning (ML) techn...,In the pursuit of sustainable energy solutions...
18,2-s2.0-85212982352,10.1016/j.ces.2024.121101,Experimental and modelling study of ammonia-ba...,Dataset of wet desulphurization scrubbing in a...,0,2-s2.0-85212982352,Experimental and modelling study of ammonia-ba...,Dataset of wet desulphurization scrubbing in a...,0,0,0,1,1,2-s2.0-85212982352,Experimental and modelling study of ammonia-ba...,Dataset of wet desulphurization scrubbing in a...,"In this work, we propose an experimental and m...",The growing energy demand and the need for mor...
24,2-s2.0-85173883804,10.1016/j.ref.2023.100490,Performance and dynamics of California offshor...,Developing reliable hourly electricity demand ...,0,2-s2.0-85173883804,Performance and dynamics of California offshor...,Developing reliable hourly electricity demand ...,0,0,1,1,1,2-s2.0-85173883804,Performance and dynamics of California offshor...,Developing reliable hourly electricity demand ...,We use statistical methods to evaluate the coi...,The global emergency to swiftly transition to ...
34,2-s2.0-105001429996,10.1016/j.sftr.2025.100578,Towards a sustainable future: The interplay of...,The KOF Globalisation Index – revisited,0,2-s2.0-105001429996,Towards a sustainable future: The interplay of...,The KOF Globalisation Index – revisited,0,1,1,1,1,2-s2.0-105001429996,Towards a sustainable future: The interplay of...,The KOF Globalisation Index – revisited,"Over the past three decades, India's transform...",Global warming and climate change due to persi...
47,2-s2.0-85126864679,10.1016/j.gloplacha.2022.103785,Linking the Mediterranean MIS 5 tephra markers...,Lake Ohrid’s tephrochronological dataset revea...,1,2-s2.0-8

In [3]:
import pandas as pd
import os

# --- 1. ファイルパスの設定 ---
GROUND_TRUTH_CSV = '../data/ground_truth/annotation_target_list.csv'
LLM_PREDICTIONS_CSV = '../data/processed/prediction_llm.csv'
SAMPLES_WITH_TEXT_CSV = '../data/processed/samples_with_text.csv'

# --- 2. データの読み込みと準備 ---
try:
    df_gt = pd.read_csv(GROUND_TRUTH_CSV)
    df_llm = pd.read_csv(LLM_PREDICTIONS_CSV)
    df_text = pd.read_csv(SAMPLES_WITH_TEXT_CSV)

    # 評価対象のモデルのカラム名
    best_model_column = 'prediction_rule3_gemini-2_5-flash'

    # 3つのファイルをDOIをキーにマージ
    df_merged = pd.merge(df_gt, df_llm, on='citing_paper_doi', how='inner', suffixes=('_gt', ''))
    df_review = pd.merge(df_merged, df_text, on='citing_paper_doi', how='inner', suffixes=('', '_text'))
    
    # -1やNaNを除外
    df_review.dropna(subset=['is_data_used_gt', best_model_column], inplace=True)
    df_review = df_review[df_review[best_model_column] != -1]
    df_review[best_model_column] = df_review[best_model_column].astype(int)
    df_review['is_data_used_gt'] = df_review['is_data_used_gt'].astype(int)
    
    # 判断の不一致を検出
    disagreements = df_review[df_review['is_data_used_gt'] != df_review[best_model_column]].copy()

except Exception as e:
    print(f"❌ エラー: データの読み込みに失敗しました。 {e}")
    disagreements = pd.DataFrame()

# --- 3. 確認用プロンプトの自動生成 ---
if not disagreements.empty:
    print("="*70)
    print(f"【確認が必要な {len(disagreements)} 件の論文について、以下のプロンプトをコピーして私に送信してください】")
    print("="*70)
    
    for index, row in disagreements.iterrows():
        human_label = "Used" if row['is_data_used_gt'] == 1 else "Not Used"
        llm_label = "Used" if row[best_model_column] == 1 else "Not Used"
        
        prompt_template = f"""
私は、ある論文が指定されたデータ論文のデータを「実際に使用しているか」を手動でラベル付けしました。
しかし、作成したAIモデルの判断と私の判断が食い違ったため、第三者の意見を参考に、私の判断が正しかったかを再確認したいです。

以下の【入力データ】と【判定基準】を基に、この論文はデータを「使用している」と判断すべきか、「使用していない」と判断すべきか、あなたの専門的な見解を教えてください。

---
### 【判定基準】
* **"Used":** 論文の主張や結論（性能評価、分析結果、比較など）を導き出すために、データセットが分析、訓練、評価、性能比較などのプロセスに直接的に用いられている状態。
* **"Not Used":** 研究の背景説明、関連研究の紹介、あるいは今後の展望としてデータセット名に言及しているだけの状態。

---
### 【入力データ】

**引用元データ論文のタイトル:**
{row['cited_data_paper_title']}

**被引用論文のタイトル:**
{row['citing_paper_title']}

**被引用論文の全文テキスト:**
{row['full_text']}

---
### 【状況】
* **私の最初の判断:** {human_label}
* **AIモデルの判断:** {llm_label}

---
### 【あなたのタスク】
上記の情報を踏まえ、最終的にどちらの判断がより妥当と考えられるか、その根拠となるテキスト中の箇所を引用しながら、あなたの分析結果を提示してください。
"""
        print("\n\n" + "#"*20 + f"  確認用プロンプト {index+1} / DOI: {row['citing_paper_doi']}  " + "#"*20)
        print(prompt_template)
        print("#"*70 + "\n")
else:
    print("✅ 人間の判断とLLMの判断はすべて一致していました。確認作業は不要です。")

【確認が必要な 35 件の論文について、以下のプロンプトをコピーして私に送信してください】


####################  確認用プロンプト 2 / DOI: 10.1016/j.jaridenv.2024.105282  ####################

私は、ある論文が指定されたデータ論文のデータを「実際に使用しているか」を手動でラベル付けしました。
しかし、作成したAIモデルの判断と私の判断が食い違ったため、第三者の意見を参考に、私の判断が正しかったかを再確認したいです。

以下の【入力データ】と【判定基準】を基に、この論文はデータを「使用している」と判断すべきか、「使用していない」と判断すべきか、あなたの専門的な見解を教えてください。

---
### 【判定基準】
* **"Used":** 論文の主張や結論（性能評価、分析結果、比較など）を導き出すために、データセットが分析、訓練、評価、性能比較などのプロセスに直接的に用いられている状態。
* **"Not Used":** 研究の背景説明、関連研究の紹介、あるいは今後の展望としてデータセット名に言及しているだけの状態。

---
### 【入力データ】

**引用元データ論文のタイトル:**
The World Checklist of Vascular Plants, a continuously updated resource for exploring global plant diversity

**被引用論文のタイトル:**
Potential effects of climate change on cacti distribution and conservation in North American drylands

**被引用論文の全文テキスト:**
Drylands biodiversity is essential to the well-being and development of approximately 2.7 billion people ( ). While drylands provide ecosystem services (e.g., nutrient cycling, primary production, water, 

In [ ]:
import pandas as pd
import os

# --- 1. 設定項目 ---
GROUND_TRUTH_CSV = '../data/ground_truth/annotation_target_list.csv'

# --- 2. 【重要】ここに修正内容を記入してください ---
# 例: corrections = {
#         "10.1016/j.jprot.2022.104578": 1, # このDOIのラベルを1に修正
#         "10.1016/j.ccell.2021.04.015": 0  # このDOIのラベルを0に修正
#     }
corrections = {
    # ↓↓↓ ここに修正したいDOIと最終的な正しいラベルを記入 ↓↓↓
    
    
    # ↑↑↑ ここまで ↑↑↑
}

# --- 3. 修正の実行と結果の確認 ---
if not corrections:
    print("修正対象が指定されていません。")
else:
    try:
        df_gt = pd.read_csv(GROUND_TRUTH_CSV)
        print(f"元の '{os.path.basename(GROUND_TRUTH_CSV)}' を読み込みました。")

        updated_count = 0
        for doi, new_label in corrections.items():
            # DOIに一致する行のインデックスを取得
            target_indices = df_gt[df_gt['citing_paper_doi'] == doi].index
            
            if not target_indices.empty:
                # is_data_used_gt列の値を新しいラベルで更新
                df_gt.loc[target_indices, 'is_data_used_gt'] = new_label
                print(f"  - DOI: {doi} のラベルを {new_label} に更新しました。")
                updated_count += 1
            else:
                print(f"  - 警告: DOI {doi} は見つかりませんでした。")

        if updated_count > 0:
            # 更新された内容をファイルに上書き保存
            df_gt.to_csv(GROUND_TRUTH_CSV, index=False, encoding='utf-8-sig')
            print(f"\n✅ {updated_count}件の修正をファイルに保存しました。")

    except Exception as e:
        print(f"\n❌ エラーが発生しました: {e}")